In [2]:
import tensorflow as tf
import zipfile
tfkl = tf.keras.layers
import csv
import numpy as np
import pandas as pd
import os as os
import matplotlib.pyplot as plt
#from keras.layers import Input, Embedding, Flatten, Dense, BatchNormalization, Dropout, concatenate
#from keras.models import Model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

# Load in Data

In [10]:
# loading in county data
data = []
ages = []
counties = []
genders = []

with open("../data/us_counties.csv", "r") as file:
    reader = csv.reader(file,delimiter=',')
    for row_index, row in enumerate(reader):
        if row_index == 0:
            print(row)
        if row_index >= 1:
            county, year, age, gender, rate = row
            year = int(year)
            if age not in ages:
                ages.append(age)
            age = ages.index(age)
            if county not in counties:
                counties.append(county)
            county = counties.index(county)
            if gender not in genders:
                genders.append(gender)
            gender = genders.index(gender)
            try:
                rate = float(rate)
            except:
                rate = -1
            if rate > 1:
                rate = 1
            # get rid of years, ages, not in health data and other cleaning
            if rate != -1:
                data.append([county, gender, year, age, rate])

county_data = np.array(data)

['fips', 'year', 'age', 'sex', 'mx']


In [11]:
print(county_data.shape)

(3569568, 5)


In [8]:
# loading in USMDB data
data = []
ages = []
states = []
genders = []

with open("../data/usmdb_5x1.csv", "r") as file:
    reader = csv.reader(file,delimiter=',')
    for row_index, row in enumerate(reader):
        if row_index == 0:
            print(row)
        if row_index >= 1:
            state, gender, year, age, rate = row
            year = int(year)
            if age not in ages:
                ages.append(age)
            age = ages.index(age)
            if state not in states:
                states.append(state)
            state = states.index(state)
            if gender not in genders:
                genders.append(gender)
            gender = genders.index(gender)
            try:
                rate = float(rate)
            except:
                rate = -1
            if rate > 1:
                rate = 1
            # get rid of years, ages, not in health data and other cleaning
            if rate != -1:
                data.append([state, gender, year, age, rate])

state_data = np.array(data)

['PopName', 'Sex', 'Year', 'Age', 'mx']


In [9]:
print(state_data.shape)
print(states)
print(genders)

(190272, 5)
['AK', 'AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
['f', 'm']


In [12]:
# loading in HMD data
data = []
ages = []
countries = []
genders = []

with open("../data/hmd_5yr.csv", "r") as file:
    reader = csv.reader(file,delimiter=",")
    for row_index, row in enumerate(reader):
        if row_index == 0:
            print(row)
        if row_index >= 1:
            year, age, country, gender, rate = row
            year = int(year)
            if age not in ages:
                ages.append(age)
            age = ages.index(age)
            if country not in countries:
                countries.append(country)
            country = countries.index(country)
            if gender not in genders:
                genders.append(gender)
            gender = genders.index(gender)
            try:
                rate = float(rate)
            except:
                rate = -1
            if rate > 1:
                rate = 1
            if rate != -1:
                data.append([country, gender, year, age, rate])

country_data = np.array(data)

['Year', 'Age', 'Country', 'Gender', 'Mortality_rate']


In [13]:
print(country_data.shape)

(182736, 5)


In [15]:
print(len(countries))

38


In [16]:
# getting unique values for geographic location column 
country_data[:,0] = country_data[:,0] + 50
county_data[:,0] = county_data[:,0] + 88

# dropping US
# country_data = country_data[country_data[:,0] != 87]
# countries.remove('USA')


In [ ]:
print(country_data.shape)

(782697, 5)


In [14]:
print(countries)

['AUS', 'AUT', 'BEL', 'BGR', 'BLR', 'CAN', 'CHE', 'CZE', 'DNK', 'ESP', 'EST', 'FIN', 'FRATNP', 'GBRTENW', 'GBR_NIR', 'GBR_SCO', 'GRC', 'HUN', 'IRL', 'ISL', 'ISR', 'ITA', 'JPN', 'LTU', 'LUX', 'LVA', 'NLD', 'NOR', 'NZL_NM', 'POL', 'PRT', 'RUS', 'SVK', 'SVN', 'SWE', 'TWN', 'UKR', 'USA']


In [17]:
combined = np.vstack((state_data, country_data, county_data))

In [18]:
print(combined.shape)

(3942576, 5)


# Train deep learning model

In [19]:
# training and test sets 
training_index = np.logical_and(combined[:, 2] >= 1959, combined[:, 2] <= 2005)
training_data = combined[training_index, :]
print(training_data.shape)

test_index = np.logical_and(combined[:, 2] > 2005, combined[:, 2] <= 2015)
test_data = combined[test_index, :]
print(test_data.shape)

final_test_index = np.logical_and(combined[:, 2] > 2015, combined[:, 2] <= 2019)
final_test = combined[final_test_index, :]
print(final_test.shape)

(2445961, 5)
(980656, 5)
(391706, 5)


In [ ]:
print(test_data)

[[0.00000e+00 0.00000e+00 2.00600e+03 0.00000e+00 8.35000e-03]
 [0.00000e+00 0.00000e+00 2.00600e+03 1.00000e+00 2.10000e-04]
 [0.00000e+00 0.00000e+00 2.00600e+03 2.00000e+00 6.30000e-04]
 ...
 [8.60000e+01 1.00000e+00 2.01300e+03 9.70000e+01 4.27027e-01]
 [8.60000e+01 1.00000e+00 2.01300e+03 9.80000e+01 4.76462e-01]
 [8.60000e+01 1.00000e+00 2.01300e+03 9.90000e+01 4.74544e-01]]


In [20]:
training_data = tf.convert_to_tensor(training_data)
test_data = tf.convert_to_tensor(test_data)
final_test = tf.convert_to_tensor(final_test)
# cast tensor to type float32
training_data = tf.cast(training_data, tf.float32)
test_data = tf.cast(test_data, tf.float32)
final_test = tf.cast(final_test, tf.float32)

num_train = training_data.shape[0]
num_test = test_data.shape[0]
num_final = final_test.shape[0]

In [ ]:
print(final_test)

tf.Tensor(
[[0.00000e+00 0.00000e+00 2.01600e+03 0.00000e+00 4.60000e-03]
 [0.00000e+00 0.00000e+00 2.01600e+03 1.00000e+00 7.40000e-04]
 [0.00000e+00 0.00000e+00 2.01600e+03 2.00000e+00 1.90000e-04]
 ...
 [8.50000e+01 1.00000e+00 2.01900e+03 9.70000e+01 2.77391e-01]
 [8.50000e+01 1.00000e+00 2.01900e+03 9.80000e+01 3.19445e-01]
 [8.50000e+01 1.00000e+00 2.01900e+03 9.90000e+01 3.32218e-01]], shape=(66400, 5), dtype=float32)


In [21]:
# define function to fetch and process data entries from training or test data 
def get_data(index, mode):
    if mode == "train":
        # randomly selects index from training data between 0 and num_train
        rand_index = tf.random.uniform([],minval=0, maxval=num_train, dtype=tf.int32) 
        entry = training_data[rand_index, :]
    elif mode == "not_random":
        # selects specified index from test data 
        entry = test_data[index, :]
    else: 
        # for any other value of mode, randomly selects index from test
        rand_index = tf.random.uniform([],minval=0, maxval=num_test, dtype=tf.int32)
        entry = test_data[rand_index, :]
    geography, gender, year, age, rate = entry[0], entry[1], entry[2], entry[3], entry[4]
    year = (year - 1998)/21
    age = tf.cast(age, tf.int32)
    geography = tf.cast(geography, tf.int32)
    gender = tf.cast(gender, tf.int32)
    year = tf.reshape(year, [1])
    age = tf.reshape(age, [1])
    geography = tf.reshape(geography, [1])
    gender = tf.reshape(gender, [1])
    rate = tf.reshape(rate, [1])
    return (year, age, geography, gender), rate

In [22]:
# use get_data function to set up training and test tensorflow datasets 
dataset_train = tf.data.Dataset.from_tensor_slices(np.arange(10000))
dataset_train = dataset_train.repeat()
dataset_train = dataset_train.map(lambda x: get_data(x, mode="train"), num_parallel_calls=4)
dataset_train = dataset_train.batch(256)
dataset_train = dataset_train.prefetch(buffer_size=512)

dataset_test = tf.data.Dataset.from_tensor_slices(np.arange(10000))
dataset_test = dataset_test.repeat()
dataset_test = dataset_test.map(lambda x: get_data(x, mode="test"), num_parallel_calls=4)
dataset_test = dataset_test.batch(256)
dataset_test = dataset_test.prefetch(buffer_size=512)

dataset_test2 = tf.data.Dataset.from_tensor_slices(np.arange(68000))
dataset_test2 = dataset_test2.map(lambda x: get_data(x, mode="not_random"), num_parallel_calls=4)
dataset_test2 = dataset_test2.batch(256)
dataset_test2 = dataset_test2.prefetch(buffer_size=512)

dataset_final = tf.data.Dataset.from_tensor_slices(np.arange(10000))
dataset_final = dataset_final.repeat()
dataset_final = dataset_final.map(lambda x: get_data(x, mode="test"), num_parallel_calls=4)
dataset_final = dataset_final.batch(256)
dataset_final = dataset_final.prefetch(buffer_size=512)

In [26]:
print(len(counties))

2472


In [27]:
def create_model():
    # defining inputs 
    year = tfkl.Input(shape=(1,), dtype='float32', name='Year')
    age =  tfkl.Input(shape=(1,), dtype='int32', name='Age')
    geography = tfkl.Input(shape=(1,), dtype='int32', name='Geography')
    gender = tfkl.Input(shape=(1,), dtype='int32', name='Gender')

    # defining embedding layers 
    age_embed = tfkl.Embedding(input_dim=24, output_dim=5, input_length=1, name='Age_embed')(age)
    age_embed = tfkl.Flatten()(age_embed)

    gender_embed = tfkl.Embedding(input_dim=2, output_dim=5, input_length=1, name='Gender_embed')(gender)
    gender_embed = tfkl.Flatten()(gender_embed)

    geography_embed = tfkl.Embedding(input_dim=2560, output_dim=5, input_length=1, name='Geography_embed')(geography)
    geography_embed = tfkl.Flatten()(geography_embed)

    # create feature vector that concatenates all inputs 
    x = tfkl.Concatenate()([year, age_embed, gender_embed, geography_embed])
    x1 = x

    # setting up middle layers 
    x = tfkl.Dense(128, activation='tanh')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(0.05)(x)

    x = tfkl.Dense(128, activation='tanh')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(0.05)(x)

    x = tfkl.Dense(128, activation='tanh')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(0.05)(x)

    x = tfkl.Dense(128, activation='tanh')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(0.05)(x)

    # setting up output layer 
    x = tfkl.Concatenate()([x1, x])
    x = tfkl.Dense(128, activation='tanh')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(0.05)(x)
    x = tfkl.Dense(1, activation='sigmoid', name='final')(x)

    # creating the model 
    model = tf.keras.Model(inputs=[year, age, geography, gender], outputs=[x])

    # compiling the model
    model.compile(loss='mse', optimizer='adam')

    return model

In [28]:
def run_deep_model(dataset_train, dataset_test):
    
    model = create_model()

    callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.25, patience=3, verbose=0, mode="auto", 
                                                    min_delta=1e-8, cooldown=0, min_lr=0.0)]
    history = model.fit(dataset_train, steps_per_epoch=1000, validation_data=dataset_test, validation_steps=500, 
                        epochs=30, verbose=2, callbacks=callbacks)

    loss_info = history.history['val_loss'][-1]

    tf.keras.backend.clear_session()

    return model, loss_info


In [29]:
model, loss_info = run_deep_model(dataset_train, dataset_test)

Epoch 1/30
1000/1000 - 19s - loss: 0.0150 - val_loss: 3.7353e-04 - lr: 0.0010 - 19s/epoch - 19ms/step
Epoch 2/30
1000/1000 - 16s - loss: 9.5943e-04 - val_loss: 1.5268e-04 - lr: 0.0010 - 16s/epoch - 16ms/step
Epoch 3/30
1000/1000 - 16s - loss: 8.0496e-04 - val_loss: 1.8149e-04 - lr: 0.0010 - 16s/epoch - 16ms/step
Epoch 4/30
1000/1000 - 17s - loss: 6.2196e-04 - val_loss: 1.6144e-04 - lr: 0.0010 - 17s/epoch - 17ms/step
Epoch 5/30
1000/1000 - 16s - loss: 4.9231e-04 - val_loss: 1.5566e-04 - lr: 0.0010 - 16s/epoch - 16ms/step
Epoch 6/30
1000/1000 - 16s - loss: 3.5222e-04 - val_loss: 1.5863e-04 - lr: 2.5000e-04 - 16s/epoch - 16ms/step
Epoch 7/30
1000/1000 - 17s - loss: 3.3397e-04 - val_loss: 1.8730e-04 - lr: 2.5000e-04 - 17s/epoch - 17ms/step
Epoch 8/30
1000/1000 - 16s - loss: 3.5400e-04 - val_loss: 1.4268e-04 - lr: 2.5000e-04 - 16s/epoch - 16ms/step
Epoch 9/30
1000/1000 - 16s - loss: 3.2209e-04 - val_loss: 1.2129e-04 - lr: 2.5000e-04 - 16s/epoch - 16ms/step
Epoch 10/30
1000/1000 - 17s - loss

In [ ]:
print(loss_info)

0.00016744744789320976


# State-Specific MSE from Combined Model

In [30]:
state_test_index = np.logical_and(state_data[:, 2] > 2005, state_data[:, 2] <= 2015)
state_test_data = state_data[state_test_index, :]
print(state_test_data.shape)

(24000, 5)


In [31]:
state_test_data = tf.convert_to_tensor(state_test_data)
# cast tensor to type float32
state_test_data = tf.cast(state_test_data, tf.float32)

In [32]:
# define function to fetch and process data entries from training or test data 
def get_state_data(index, mode):
    if mode == "train":
        # randomly selects index from training data between 0 and num_train
        rand_index = tf.random.uniform([],minval=0, maxval=num_train, dtype=tf.int32) 
        entry = training_data[rand_index, :]
    elif mode == "not_random":
        # selects specified index from test data 
        entry = state_test_data[index, :]
    else: 
        # for any other value of mode, randomly selects index from test
        rand_index = tf.random.uniform([],minval=0, maxval=num_test, dtype=tf.int32)
        entry = test_data[rand_index, :]
    geography, gender, year, age, rate = entry[0], entry[1], entry[2], entry[3], entry[4]
    year = (year - 1998)/21
    age = tf.cast(age, tf.int32)
    geography = tf.cast(geography, tf.int32)
    gender = tf.cast(gender, tf.int32)
    year = tf.reshape(year, [1])
    age = tf.reshape(age, [1])
    geography = tf.reshape(geography, [1])
    gender = tf.reshape(gender, [1])
    rate = tf.reshape(rate, [1])
    return (year, age, geography, gender), rate

In [35]:
dataset_state_test = tf.data.Dataset.from_tensor_slices(np.arange(7000))
dataset_state_test = dataset_state_test.map(lambda x: get_state_data(x, mode="not_random"), num_parallel_calls=4)
dataset_state_test = dataset_state_test.batch(256)
dataset_state_test = dataset_state_test.prefetch(buffer_size=512)

In [36]:
predictions = model.predict(dataset_state_test)

28/28 [==============================] - 0s 4ms/step


In [37]:
# Get the true values from the test dataset
true_values = []
for _, rate in dataset_state_test:
    true_values.extend(rate.numpy())

# Convert true_values to a numpy array
true_values = np.array(true_values)

# Convert predictions to a numpy array if not already
predictions = np.array(predictions)

# Compute MSE using TensorFlow
mse = np.mean((true_values - predictions)**2)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.0001929712


# Country-Specific MSE

In [ ]:
country_test_index = np.logical_and(country_data[:, 2] > 2005, country_data[:, 2] <= 2015)
country_test_data = country_data[country_test_index, :]
print(country_test_data.shape)

(72000, 5)


In [ ]:
country_test_data = tf.convert_to_tensor(country_test_data)
# cast tensor to type float32
country_test_data = tf.cast(country_test_data, tf.float32)

In [ ]:
# define function to fetch and process data entries from training or test data 
def get_country_data(index, mode):
    if mode == "train":
        # randomly selects index from training data between 0 and num_train
        rand_index = tf.random.uniform([],minval=0, maxval=num_train, dtype=tf.int32) 
        entry = training_data[rand_index, :]
    elif mode == "not_random":
        # selects specified index from test data 
        entry = country_test_data[index, :]
    else: 
        # for any other value of mode, randomly selects index from test
        rand_index = tf.random.uniform([],minval=0, maxval=num_test, dtype=tf.int32)
        entry = test_data[rand_index, :]
    geography, gender, year, age, rate = entry[0], entry[1], entry[2], entry[3], entry[4]
    year = (year - 1998)/21
    age = tf.cast(age, tf.int32)
    geography = tf.cast(geography, tf.int32)
    gender = tf.cast(gender, tf.int32)
    year = tf.reshape(year, [1])
    age = tf.reshape(age, [1])
    geography = tf.reshape(geography, [1])
    gender = tf.reshape(gender, [1])
    rate = tf.reshape(rate, [1])
    return (year, age, geography, gender), rate

In [ ]:
dataset_country_test = tf.data.Dataset.from_tensor_slices(np.arange(72000))
dataset_country_test = dataset_country_test.map(lambda x: get_country_data(x, mode="not_random"), num_parallel_calls=4)
dataset_country_test = dataset_country_test.batch(256)
dataset_country_test = dataset_country_test.prefetch(buffer_size=512)

In [ ]:
predictions = model.predict(dataset_country_test)

282/282 [==============================] - 2s 5ms/step


In [ ]:
# Get the true values from the test dataset
true_values = []
for _, rate in dataset_country_test:
    true_values.extend(rate.numpy())

# Convert true_values to a numpy array
true_values = np.array(true_values)

# Convert predictions to a numpy array if not already
predictions = np.array(predictions)

# Compute MSE using TensorFlow
mse = tf.reduce_mean(tf.square(true_values - predictions)).numpy()
print("Mean Squared Error:", mse)

Mean Squared Error: 0.0002767175
